# VacationPy

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

# Store Part I results into DataFrame
- Load the csv exported in Part I to a DataFrame

In [2]:
# Import cities csv file
path = "../output_data/cities.csv"
cities = pd.read_csv(path)
cities.head()

,City,Country,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Date
0,Ushuaia,AR,-54.8000,-68.3000,53.60,66,75,5.75,1613084207
1,Saint-Philippe,RE,-21.3585,55.7679,78.80,61,99,4.61,1613084449
2,Barrow,US,71.2906,-156.7887,-43.60,83,1,9.22,1613084247
3,Mataura,NZ,-46.1927,168.8643,61.59,49,1,2.19,1613084389
4,Punta Arenas,CL,-53.1500,-70.9167,55.40,47,0,31.07,1613084545


# Humidity Heatmap

- Configure gmaps.
- Use the Lat and Lng as locations and Humidity as the weight
- Add Heatmap layer to map.

In [3]:
# Config gmaps
gmaps.configure(api_key=g_key)

# Based on locations and humidity
locations = cities[['Latitude', 'Longitude']]
humidity = cities["Humidity (%)"]

# Figure gmaps
fig = gmaps.figure(map_type="HYBRID")

# Create heat layer with gmaps using locations and humidity
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(cities["Humidity (%)"]),
                                 point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

# Create new DataFrame fitting weather criteria

- Narrow down the cities to fit weather conditions.
- Drop any rows will null values.

In [4]:
# Create vacation dataframe.  My dream location.  Little clouds, barely any wind.  Temperature between 85-70
vacation_df= cities.loc[(cities['Max Temperature (F)']<=85) & (cities['Max Temperature (F)']>=70) 
                        & (cities['Wind Speed (mph)']<=5) 
                        & (cities['Cloudiness (%)']<=10)]

# Drop null values
vacation_df = vacation_df.dropna()

# Print df
vacation_df

,City,Country,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Date
166,Taltal,CL,-25.4000,-70.4833,70.00,63,0,4.74,1613084792
190,Luanda,AO,-8.8368,13.2343,80.60,78,0,4.61,1613084175
212,Makkah al Mukarramah,SA,21.4267,39.8261,74.55,59,0,2.93,1613084798
288,Negombo,LK,7.2083,79.8358,71.60,78,0,4.61,1613084761
369,Carnot,CF,4.9409,15.8665,72.23,22,3,3.44,1613084715
377,Kanel,SN,15.4916,-13.1763,82.40,24,0,2.30,1613084820
423,Salalah,OM,17.0151,54.0924,71.60,53,0,3.44,1613084482
469,Viedma,AR,-40.8135,-62.9967,77.00,55,0,1.01,1613084448
505,Posadas,AR,-27.3671,-55.8961,80.60,57,0,3.44,1613084839
532,Buckeye,US,33.3703,-112.5838,77.00,21,1,3.24,1613084772


# Hotel Map

- Store into variable named hotel_df.
- Add a "Hotel Name" column to the DataFrame.
- Set parameters to search for hotels with 5000 meters.
- Hit the Google Places API for each city's coordinates.
- Store the first Hotel result into the DataFrame.
- Plot markers on top of the heatmap.

In [5]:
# Rename vacation df to hotel_df.  Add hotel name column
hotel_df = vacation_df
hotel_df["Hotel Name"] = ""

# Create params
target_radius = 5000
target_type = "lodging"
gkey = g_key
keyword = "hotel"

# Set params
params = {"radius": target_radius,
    "type": target_type,
    "key": gkey,
      "keyword" : keyword}

# Generate hotels and store in hotel_df
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
    # convert to json
    hotel_name = hotel_name.json()
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]

    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [6]:
# Visualize to confirm hotel name appears
hotel_df

,City,Country,Latitude,Longitude,Max Temperature (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Date,Hotel Name
166,Taltal,CL,-25.4000,-70.4833,70.00,63,0,4.74,1613084792,Hostal Beatriz
190,Luanda,AO,-8.8368,13.2343,80.60,78,0,4.61,1613084175,EPIC SANA Luanda
212,Makkah al Mukarramah,SA,21.4267,39.8261,74.55,59,0,2.93,1613084798,Pullman Zamzam Makkah
288,Negombo,LK,7.2083,79.8358,71.60,78,0,4.61,1613084761,Pledge Scape Negombo
369,Carnot,CF,4.9409,15.8665,72.23,22,3,3.44,1613084715,
377,Kanel,SN,15.4916,-13.1763,82.40,24,0,2.30,1613084820,
423,Salalah,OM,17.0151,54.0924,71.60,53,0,3.44,1613084482,InterCityHotel Salalah
469,Viedma,AR,-40.8135,-62.9967,77.00,55,0,1.01,1613084448,Grupo Costanera (Departamento San Gabriel)
505,Posadas,AR,-27.3671,-55.8961,80.60,57,0,3.44,1613084839,Carnaval Hotel Casino
532,Buckeye,US,33.3703,-112.5838,77.00,21,1,3.24,1613084772,Westward Motel


In [7]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [8]:
# Add marker layer ontop of heat map
gmaps.configure(api_key=g_key)

fig = gmaps.figure(map_type="HYBRID")


markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add layer
fig.add_layer(markers)
fig.add_layer(heat_layer)

# Display figure
fig


Figure(layout=FigureLayout(height='420px'))